<a href="https://colab.research.google.com/github/FrancoNMarquez/CHAT-LEF/blob/master/TF_Costos_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Sacar Costo para hacer el embeddings, Preparar indices y subirlo Pinecone**


In [ ]:
!pip install openai -q
!pip install langchain -q
!pip install tiktoken -q
!pip install pinecone-client -q
#!pip install pinecone-client --upgrade!
!pip install python-dotenv -q
!pip install langchain_pinecone ## Subir a pinecone los vectores


In [2]:
import os
from google.colab import drive
# Montar Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
 # Carga OPENAI_API_KEY
from dotenv import load_dotenv, find_dotenv
load_dotenv('/content/drive/MyDrive/Archivos/NLP/.env')
#os.environ.get("OPENAI_API_KEY")


True

In [6]:
# Leer el txt contenido_combinado.tx para  extraer el txt en el formato que reconoce
from langchain_community.document_loaders import TextLoader
#loader = TextLoader("/content/drive/MyDrive/chatlef/Txt/Modificado/FINAL/contenido_combinado.txt")
loader = TextLoader("/content/drive/MyDrive/chatlef/Videos.txt")

documento = loader.load()

In [55]:
# Leer los fragmentos del archivo
# No funciona cuando lo extremos como un archivo texto al parecer necesita la estructura de datos documento[0].page_content ect...
#with open("/content/drive/MyDrive/chatlef/Txt/contenido_unido.txt", "r", encoding="utf-8") as file:
#    documento = file.readlines()

In [7]:
# Mostramos algunos datos
print(documento)
print(len(documento[0].page_content))

[Document(page_content='Clase1 (parte 1) https://www.youtube.com/watch?v=zPw_xqn_Ztk  Introducción a conceptos básicos de IA \nClase1 (parte 2) https://www.youtube.com/watch?v=UXxeAYfkdkw  Herramientas a utilizar en el curso.    \nClase2 (parte 1) https://www.youtube.com/watch?v=uIrEkVyrFT0  ACCEDIENDO A CONJUNTOS DE DATOS \nClase2 (parte 2) https://www.youtube.com/watch?v=qL_8Un-C2YI  LIMPIEZA DE DATOS \nClase3 (parte 1) https://www.youtube.com/watch?v=XVjzqtohGKM  OPERACIONES DE MANEJO DE DATOS "DATA WRANGLING"\nClase3 (parte 2) https://www.youtube.com/watch?v=6Npzfhjrkb4  OPERACIONES DE MANEJO DE DATOS "DATA WRANGLING"\nClase4 (parte 2) https://www.youtube.com/watch?v=O_VUT1zJRyQ  OPERACIONES DE MANEJO DE DATOS "DATA WRANGLING" (2da parte)\nClase5 (parte 1) https://www.youtube.com/watch?v=RJUwGLygrOQ  JOIN DE DATOS\nClase5 (parte 2) https://www.youtube.com/watch?v=SOvyzc_pwoI  WEB SCRAPING\nClase6 (parte 1) https://www.youtube.com/watch?v=wHbg-O1P-Nc  Regresión Lineal (teoría) \nCla

In [8]:
# Fragmentacion
from langchain.text_splitter import RecursiveCharacterTextSplitter as RC
text_splitter=RC(chunk_size=200,chunk_overlap=20,length_function=len)
fragmentos=text_splitter.split_documents(documento)
print(len(fragmentos))

48


In [9]:
fragmentos[40].page_content

'Clase30 (parte 1) https://www.youtube.com/watch?v=eEO4d-c15lU LANGCHAIN - Sistema de respuesta con base de datos propia en PDF'

In [10]:
# Para sacar el presupuesto
def print_embedding_cost(texts):
    import tiktoken
    # Obtener el codificador para el modelo de embedding
    enc = tiktoken.encoding_for_model("text-embedding-ada-002")
    # Calcular el número total de tokens para todos los textos
    total_tokens = sum([len(enc.encode(text)) for text in texts])
    # Calcular el costo estimado del embedding (suponiendo $0.0001 por cada 1000 tokens)
    embedding_cost = (total_tokens / 1000) * 0.0001
    # Imprimir el costo total y el costo por token
    print(f"Total Tokens: {total_tokens}")
    print(f"Embedding Cost (USD): {embedding_cost:.5f}")

In [11]:

# Calcular el costo del embedding de los fragmentos
# aca se complico por que no me toma el array de fragmentos, entonces decidi sacarlo del documento directamente el costo seria mayor por el overlap
print_embedding_cost(documento[0].page_content)

Total Tokens: 6663
Embedding Cost (USD): 0.00067


In [12]:
# Conectarse con Pinecone
from pinecone import Pinecone, PodSpec
pc = Pinecone(api_key=os.environ.get("PINECONE_API_KEY"))

In [ ]:
pc.list_indexes()

In [26]:
pc.list_indexes().names()

['videos']

In [ ]:
pc.describe_index("videos")

In [ ]:
for index in pc.list_indexes():
    print(index.name)
    print(index.dimension)
    print(index.metric)
    print(index.status)
    print(index.host)
    print(index.spec)

In [73]:
#Borrar todos los indices
for i in pc.list_indexes():
    #print(i.name)
    pc.delete_index(i.name)

In [74]:
# Crear Indice usar (spec=PodSpec) cambio la forma de crearlo
indice="videos"
if indice not in pc.list_indexes().names():
    pc.create_index(
      name=indice,
      dimension=1536,
      metric="cosine",
      spec=PodSpec(
        environment="gcp-starter",
        pod_type="p1.x1",
        pods=1
      )
    )

Crear Embeddings

In [15]:
from langchain.embeddings import OpenAIEmbeddings
#embeddings=OpenAIEmbeddings()
model_name = 'text-embedding-ada-002'
OPENAI_API_KEY=os.environ.get("OPENAI_API_KEY")
embeddings= OpenAIEmbeddings(
    model=model_name,
    openai_api_key=OPENAI_API_KEY
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [76]:
## Subir a pinecone los vectores
from langchain_pinecone import Pinecone
index_name = "videos"
docsearch = Pinecone.from_documents(fragmentos, embeddings, index_name=index_name)

In [20]:
fragmentos[1].page_content

'Clase1 (parte 2) https://www.youtube.com/watch?v=UXxeAYfkdkw  Herramientas a utilizar en el curso.    \nClase2 (parte 1) https://www.youtube.com/watch?v=uIrEkVyrFT0  ACCEDIENDO A CONJUNTOS DE DATOS'

In [39]:
# Agregar vectores a un indice existente En caso que querramos incorporar mas datos a nuestro embedding
from langchain_pinecone import Pinecone
index_name = "videos"
vectorstore = Pinecone(index_name=index_name, embedding=embeddings)
texts = [doc.page_content for doc in fragmentos]
vectorstore.add_texts(texts)


['1249de53-fe3d-467a-93f1-a793243854cf',
 'c881fbeb-7709-44b8-bbee-8e1a5d3c93ba',
 '073abd64-3891-426d-9479-9af85596b7c8',
 'd17d210b-9610-4c7e-843b-8146688bd271']

In [ ]:
texts

Documentacion importante

 https://python.langchain.com/docs/integrations/vectorstores/pinecone
